# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,0.60,83,69,4.34,GS,1715035486
1,1,gadzhiyevo,69.2551,33.3362,-2.81,62,100,7.54,RU,1715035486
2,2,ancud,-41.8697,-73.8203,6.57,94,97,1.05,CL,1715035454
3,3,island harbour,50.7240,-1.2758,13.43,73,100,1.79,GB,1715035487
4,4,ta`u,-14.2336,-169.5144,28.67,81,100,4.15,AS,1715035487


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM',
    size='Humidity',
    color='blue',
    hover_cols=['City', 'Country', 'Humidity'],
    title='Cities and their Humidity'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,waddan,29.1614,16.1390,22.74,25,0,4.01,LY,1715035489
84,84,nova vicosa,-17.8919,-39.3719,25.35,83,0,4.07,BR,1715035414
99,99,belmonte,-15.8631,-38.8828,25.31,84,0,3.27,BR,1715035422
150,150,morondava,-20.2833,44.2833,23.05,58,0,3.32,MG,1715035506
155,155,saint-pierre,-21.3393,55.4781,21.82,68,0,3.60,RE,1715035395


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
21,waddan,LY,29.1614,16.1390,25,
84,nova vicosa,BR,-17.8919,-39.3719,83,
99,belmonte,BR,-15.8631,-38.8828,84,
150,morondava,MG,-20.2833,44.2833,58,
155,saint-pierre,RE,-21.3393,55.4781,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    "categories": "accommodation.hotel", 
    "apiKey": geoapify_key, 
    "limit": 1  # limit the results to 1 to get the nearest hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waddan - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
belmonte - nearest hotel: No hotel found
morondava - nearest hotel: Hotel Sharon Menabe
saint-pierre - nearest hotel: Tropic Hotel
east london - nearest hotel: No hotel found
beira - nearest hotel: Hotel Savoy
west end - nearest hotel: The Inn at Old Bahama Bay
la foa - nearest hotel: Oasis de Tendea
port saint john's - nearest hotel: Outback Inn
jejuri - nearest hotel: hotel durga lunch home
piracaia - nearest hotel: No hotel found
plymouth - nearest hotel: Rodeway Inn Middleboro-Plymouth
gebeit - nearest hotel: No hotel found
bulungu - nearest hotel: No hotel found
maintirano - nearest hotel: Hotel Safari


,City,Country,Lat,Lng,Humidity,Hotel Name
21,waddan,LY,29.1614,16.1390,25,No hotel found
84,nova vicosa,BR,-17.8919,-39.3719,83,Pousada Pontal da Barra
99,belmonte,BR,-15.8631,-38.8828,84,No hotel found
150,morondava,MG,-20.2833,44.2833,58,Hotel Sharon Menabe
155,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
276,east london,ZA,-33.0153,27.9116,52,No hotel found
285,beira,MZ,-19.8436,34.8389,88,Hotel Savoy
303,west end,BS,26.6833,-78.9667,69,The Inn at Old Bahama Bay
336,la foa,NC,-21.7108,165.8276,60,Oasis de Tendea
364,port saint john's,ZA,-31.6229,29.5448,47,Outback Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM',
    size='Humidity',
    color='blue',
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    title='Cities with Nearest Hotels'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)